In [1]:
import pandas as pd
import numpy as np
from jupDeep import *
import hashlib


'''
    we drop the columns we are not working with
'''
df = pd.read_csv('dataBlock_new.csv')
df = df.drop(['elementHeight', 'elementWidth','rap_size'], axis=1)


# df.head()

In [7]:
w = 27
h = 27

'''
    which columns will be divided by how much. If the column has the value None, it will not be divided
'''
col_div = {
           'globalBlockHeight' : h, 
           'x_orig': w, 
           'y_orig': h,
           'x_end': w,
           'y_end': h
          }

df_new = df.copy()

for col_key in col_div:
    if col_div[col_key] != None:
        df_new[col_key] = df_new[col_key]/col_div[col_key]
        df_new[col_key] = df_new[col_key].apply(np.floor)

df_new.head()
theme_uniq = [ele for ele in df['theme'].unique()]
params_uniq = ['x_orig','y_orig', 'x_end', 'y_end', 'type'] # modifica dupa necesitate globalBlockHeight
infoBlocks = {}

for theme in theme_uniq:
    t = df_new[df_new['theme'] == theme]
    bl = [ele for ele in t['block'].unique()]

    for block in bl:
        block_item = t[t['block']== block][params_uniq]
        block_item = block_item.values.tolist()
        block_item = np.concatenate(block_item).tolist()
        block_item = ''.join(str(block_item))
        way = str(theme) + "/" + str(block)
        infoBlocks[way] = hashlib.sha256(block_item.encode('utf-8')).hexdigest()

flipped = {}
for key, value in infoBlocks.items():
    if value not in flipped:
        flipped[value] = [key]
    else:
        flipped[value].append(key)

res_hash = []
count = 0
for hs in flipped:
    if len(flipped[hs])>1:
        count += 1
        res_hash += [flipped[hs]]


print('total = {}'.format(len(res_hash)))
# # dd(res_hash)
# df_new.head()

total = 27


In [16]:
import math

name_blocks0 = []
name_blocks1 = []

dist = []
coincidence = []
coincidence_per_dot = []
no_coincidence = []

pp = ['x_orig','y_orig', 'x_end', 'y_end']

for pare in res_hash:
    if len(pare) == 2:
        name_blocks0 += [pare[0]]
        way0 = pare[0].split('/')
        theme0 = way0[0]
        block0 = way0[1]

        name_blocks1 += [pare[1]]
        way1 = pare[1].split('/')
        theme1 = way1[0]
        block1 = way1[1]

        cond_0 =  (df['theme']==theme0) & (df['block']== block0)
        cond_1 =  (df['theme']==theme1) & (df['block']== block1)

        data0 = df.loc[cond_0, pp ]
        data1 = df.loc[cond_1, pp ]

        d = (((np.array(data0['x_orig']) - np.array(data1['x_orig']))**2 + (np.array(data0['y_end']) - np.array(data1['y_end']))**2).sum())**0.5
        dist += [round(d, 1)]

        cnd = (data0.reset_index(drop=True) == data1.reset_index(drop=True)).to_numpy()
        cnd = np.concatenate(cnd)
        cnd_s = cnd.sum()
        cnd_c = len(cnd)
        dif_c_s = cnd_c-cnd_s
        no_coincidence += [dif_c_s]

        coincidence += [round(100*cnd_s/cnd_c, 1)]
        if dif_c_s != 0:
            coincidence_per_dot += [round(d/dif_c_s, 1)]
        else:
            coincidence_per_dot += ['-']

    else:
        print('in acest set sunt mai multe blockuri decit 2 :\n', pare)

df_res = pd.DataFrame ({'name_block 0' : name_blocks0, 'name_block 1' : name_blocks1, 'total diff coord' : dist, 'mean diff coord per param' : coincidence_per_dot, 
                       'no_coincidence' : no_coincidence,  'coincidence %' : coincidence} )

# for index, elem in df_res.iterrows():
#     print(elem)

df_res = df_res.sort_values(by=[ 'coincidence %', 'mean diff coord per param','total diff coord'])
print('numarul total : {}'.format(len(df_res)))
df_res.head(50)
df_res.to_csv('list 0 - 27 - 27 buc.csv', index=False)

numarul total : 27


In [11]:
import subprocess

path = 'same_blocks'
mkdir(path)

i = 1
for blocks in res_hash:
    way = path + '/' + str(i)
    mkdir(way)
    if len(blocks) == 2:

        for blk in blocks:
            src = '/'.join(['blocks', blk])
            src = src.replace("__", ",_")

            blk_dst = blk.split('/')
            blk_dst = '---'.join(blk_dst)
            blk_dst = blk_dst.replace("__", ",_")


            dst = '/'.join([way, blk_dst])
            command = 'cp {} {}'.format(src, dst)

            subprocess.call(command, shell=True)

    i += 1
    
print('end code ...')


end code ...


In [ ]:
type_uniq = [ele for ele in df['type'].unique()]
cond_1 = (df['block']=='71_links.json')  & (df['theme']=='verso-local-blocks')
cond_2 = (df['block']=='2_images_with_text_left_and_text_with_call_to_action_right.json')  & (df['theme']=='verso-blocks')


da = df.loc[cond_1 | cond_2, params_uniq + ['theme','block']]
da.head(50)

In [10]:
cond_0 = (df['theme']=='verso-local-blocks') & (df['block']=='71_links.json')  
cond_1 = (df['theme']=='verso-blocks') & (df['block']=='2_images_with_text_left_and_text_with_call_to_action_right.json')

name_blocks = ['71_links.json $$$ 2_images_with_text_left_and_text_with_call_to_action_right.json']
pp = ['x_orig','y_orig', 'x_end', 'y_end']
data0 = df.loc[cond_0, pp ]
data1 = df.loc[cond_1, pp ]

dist = []
coincidence = []


print(data0, '\n\n',data1)

d = (((np.array(data0['x_orig']) - np.array(data1['x_orig']))**2 + (np.array(data0['y_end']) - np.array(data1['y_end']))**2).sum())**0.5
dist += [d]

# print(dist)
dim_df = len(data0)*len(params_uniq)
cnd = (data0.reset_index(drop=True) == data1.reset_index(drop=True)).to_numpy()

cnd = np.concatenate(cnd)
cnd_s = cnd.sum()
cnd_c = len(cnd)

# print(cnd_s, cnd_c)

coincidence += [round(100*cnd_s/cnd_c, 1)]

df_fe = pd.DataFrame({'name_block' : name_blocks, 'diff_coord' : dist, 'coincidence %' : coincidence} )

df_fe.head()

      x_orig  y_orig   x_end  y_end
4426   853.0   568.0  1043.0  618.0
4427   293.0    79.0   713.0  700.0
4428    90.0   401.0   230.0  608.0
4429   853.0   397.0  1098.0  509.0
4430   853.0   200.0  1098.0  344.0
4431    90.0   200.0   230.0  280.0
4432    90.0   161.0   153.0  205.0 

       x_orig  y_orig   x_end  y_end
7499   853.0   568.0  1043.0  618.0
7500   280.0    79.0   713.0  700.0
7501    90.0   401.0   230.0  608.0
7502   853.0   397.0  1098.0  509.0
7503   853.0   200.0  1098.0  344.0
7504    90.0   200.0   230.0  280.0
7505    90.0   161.0   153.0  205.0


,name_block,diff_coord,coincidence %
0,71_links.json $$$ 2_images_with_text_left_and_...,13.0,96.4
